In [ ]:
# main.py
#from data_preparation import load_and_preprocess_image
from image_processing import load_and_preprocess_image, load_and_preprocess_image_without_selective
from model_utils import save_history_to_excel, plot_convergence_curves, plot_confusion_matrix
#from optimizers.HGS_optimizer import optimize_hyperparameters
from optimizers.HGS_optimizer import HGS as hgs
#from models.vgg16_model import construct_model
#from trainers.model_trainer import train_model, test_model



from keras.applications.vgg16 import preprocess_input, decode_predictions

from sklearn.model_selection import train_test_split

#import the OpenCV library in Python, specifically the cv2 module, The cv2 module contains functions for image processing and computer vision tasks.
import os, cv2, keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical #Thaer

from keras.applications.vgg16 import VGG16
# from PIL import Image
import matplotlib.pyplot as plt #thaer
import pandas as pd #thaer
from google.colab import drive
import sys
import pickle as pkl
sys.path.append('/')

#import selector as slctr
import pandas as pd
import fitnessFUN
import time


# Select number of repetitions for each experiment.
NumOfRuns=1

# Select general parameters for all optimizers (population size, number of iterations)
PopulationSize = 5
Iterations= 10

#Export results ?
Export=True

#Automaticly generated file name by date and time
ExportToFile="experiment"+time.strftime("%Y-%m-%d-%H-%M-%S")+".csv"

# Check if it works at least once
Flag=False

# CSV Header for for the convergence
CnvgHeader1=[]
CnvgHeader2=[]


for l in range(0,Iterations):
	CnvgHeader1.append("Iter"+str(l+1))

for l in range(0,Iterations):
	CnvgHeader2.append("Iter"+str(l+1))

#++++++++++++++++++++++++++++++++++++  Data loading and preprocessing +++++++++++++++++++++++++++++++++
annot= "flickr_logos_27_dataset_original/flickr_logos_27_dataset_training_set_annotation.txt"
path = "flickr_logos_27_dataset_original/flickr_logos_27_dataset_images"
base_path = "flickr_logos_27_dataset_original/"
#/content/
# img = cv2.imread('images/test.jpeg')
cv2.setUseOptimized(True)   #tells OpenCV to use optimized code whenever possible, which can lead to better performance for many operations
cv2.setNumThreads(8)  # This function sets the number of threads that OpenCV will use for parallel operations

#train_images, train_labels, train_labelsnum, train_labelsnumSet = load_and_preprocess_image(path, annot)
train_images, train_labels, train_labelsnum, train_labelsnumSet = load_and_preprocess_image_without_selective(path, annot)

yarr = np.array(train_labelsnum)

for k in range (0,NumOfRuns):
    # Split data into training and testing
    train_x, test_x, train_y, test_y = train_test_split(train_images, yarr, test_size=0.25)
    # Convert labels to categorical
    train_labels = to_categorical(train_y, num_classes=len(train_labelsnumSet))
    # test_labels = to_categorical(test_y, num_classes=8)

    y_train = to_categorical(train_y, len(train_labelsnumSet))
    y_test = to_categorical(test_y, len(train_labelsnumSet))

    train_ds = train_x
    test_ds = test_x

    #============ checkpoint =====================
    # Assuming train_images is a list of numpy arrays
    train_ds = tf.convert_to_tensor(train_ds)
    test_ds = tf.convert_to_tensor(test_ds)
    print("the length is : ", len(train_labelsnumSet))
    print("the data is : ", train_labelsnumSet)

    #to save it
    with open("lbldata.txt", 'w') as f:
        for s in train_labelsnumSet:
            f.write(str(s) + '\n')

    # Define lower and upper bounds for each hyperparameter
    lb = [0.0001, 128, 0, 0, 3, 16]  # Lower bounds
    ub = [0.01, 4096, 2, 2, 10, 128] # Upper bounds   or  ub = [0.01, 4096, 2, 2, 10, 128]

    dim = len(lb)
    function_name = "objective_function"
    # Call HGS

    x=hgs(getattr(fitnessFUN, function_name), lb, ub, dim, PopulationSize, Iterations, args=(train_ds, train_labels, train_labelsnumSet))

    if(Export==True):
        with open(ExportToFile, 'a',newline='\n') as out:
            writer = csv.writer(out,delimiter=',')
            if (Flag==False): # just one time to write the header of the CSV file
                header= numpy.concatenate([["Optimizer","Dataset","objfname","Experiment","startTime","EndTime","ExecutionTime","trainAcc","testAcc"],CnvgHeader1,CnvgHeader1])
                writer.writerow(header)
            a=numpy.concatenate([['HGS',"Flicker27",x.objfname,k+1,x.startTime,x.endTime,x.executionTime,"50","50"],x.convergence1,x.convergence2])
            writer.writerow(a)
        out.close()
    Flag=True # at least one experiment

if (Flag==False): # Faild to run at least one experiment
    print("No Optomizer or Cost function is selected. Check lists of available optimizers and cost functions")

    # Optimize hyperparameters (This is where PSO optimization would be invoked)
    #optimized_hyperparameters = optimize_hyperparameters(train_images, train_labels, train_labelsnumSet)

#def main():
    # Data loading and preprocessing ++++++++++++++++++++++++++++++
    #path = "flickr_logos_27_dataset_original/flickr_logos_27_dataset_images"
    #annot = "flickr_logos_27_dataset_original/flickr_logos_27_dataset_training_set_annotation.txt"
    #train_images, train_labels, train_labelsnum, train_labelsnumSet = load_and_preprocess_image(path, annot)

    # Convert labels to categorical ++++++++++++++++++++++++++++++++
    #train_labels = np.array(train_labelsnum)
    #train_labels = to_categorical(train_labels, num_classes=len(train_labelsnumSet))

    # Split data into training and testing ++++++++++++++++++++++++
    # Assuming you have a utility function for this or use train_test_split as before
    # train_x, test_x, train_y, test_y = ...

    # Optimize hyperparameters (This is where PSO optimization would be invoked) +++++++++++++++++
    #optimized_hyperparameters = optimize_hyperparameters(train_images, train_labels, train_labelsnumSet)

    # Construct, train, and evaluate the model using the optimized hyperparameters
    # model = construct_model(...)
    # history = train_model(...)
    # test_model(...)

#if __name__ == "__main__":
#    main()
